In [1]:
#error analysis week5
import os
current_dir = os.getcwd()
if current_dir.endswith("week_4_5"):
    os.chdir("..")
    print(os.getcwd())
else:
    print("current dir", current_dir)
    
import pandas as pd
preds_labels_no_finetune_df = pd.read_csv('week_4_5/data/eval_results_meta_llama_no_finetune.csv')
preds_labels_finetuned_df = pd.read_csv('week_4_5/data/eval_results_axo-2024-09-24-20-47-06-872a.csv')


answerable_idxs = preds_labels_no_finetune_df['answer'] != 'no'
non_answerable_idxs = preds_labels_no_finetune_df['answer'] == 'no'



/Users/HansPeter/Dev/ku_local/nlp_course/src/nlp_course/code


In [2]:
preds_labels_no_finetune_df.columns

Index(['answer', 'guess'], dtype='object')

In [3]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score


#no finetune 

no_finetune_labels = preds_labels_no_finetune_df['answer']
no_finetune_preds = preds_labels_no_finetune_df['guess']

finetuned_labels = preds_labels_finetuned_df['answer']
finetuned_preds = preds_labels_finetuned_df['guess']



In [4]:
from week_4_5.utils import exact_match, f1_score

def get_perf(labels, preds):
    em, em_idxs = exact_match(preds, labels)
    f1, f1_idxs = f1_score(preds, labels)
    return {'exact_match': em, 'f1_score': f1, 'exact_match_idxs': em_idxs, 'f1_scores': f1_idxs}

#perf non answerable

perf_non_finetune_non_answerable = get_perf(no_finetune_labels[non_answerable_idxs], no_finetune_preds[non_answerable_idxs])
perf_finetuned_non_answerable = get_perf(finetuned_labels[non_answerable_idxs], finetuned_preds[non_answerable_idxs])

perf_non_finetune_answerable = get_perf(no_finetune_labels[answerable_idxs], no_finetune_preds[answerable_idxs])
perf_finetuned_answerable = get_perf(finetuned_labels[answerable_idxs], finetuned_preds[answerable_idxs])



print('f1_score answerable no finetune', perf_non_finetune_answerable['f1_score'])
print('f1_score answerable finetuned', perf_finetuned_answerable['f1_score'])
print('exact_match answerable no finetune', perf_non_finetune_answerable['exact_match'])
print('exact_match answerable finetuned', perf_finetuned_answerable['exact_match'])


#analyze the semantic distance between the guesses and labels

/opt/miniconda3/envs/ml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


f1_score answerable no finetune 0.12948806187096643
f1_score answerable finetuned 0.5778806799387848
exact_match answerable no finetune 0.04590163934426229
exact_match answerable finetuned 0.4483606557377049


In [6]:
os.listdir()

['week4_hf_notebook.ipynb',
 'week4.ipynb',
 'results_log_week4.txt',
 '.DS_Store',
 'week2.ipynb',
 'week3.ipynb',
 '__init__.py',
 '__pycache__',
 'week1.ipynb',
 '.last_run_name',
 'week_4_5',
 'main.py']

In [12]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

f1_scores_finetuned = perf_finetuned_answerable['f1_scores']
f1_scores_no_finetune = perf_non_finetune_answerable['f1_scores']

fig = make_subplots(rows=1, cols=2, subplot_titles=['llama3-1-8B-instruct-finetuned', 'llama3-1-8B-no-finetuned'])

fig.add_trace(
    go.Histogram(x=f1_scores_finetuned, nbinsx=100, opacity=0.7, name='Finetuned'),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x=f1_scores_no_finetune, nbinsx=100, opacity=0.7, name='No finetuned'),
    row=1, col=2
)

fig.update_layout(showlegend=False)
fig.update_layout(title='F1 scores for answerable questions for finetuned model and no finetuned model')
fig.show()

import os
current_dir = os.getcwd()
if current_dir.endswith("week_4_5") or current_dir.endswith("code"):
    os.chdir("..")
    print(os.getcwd())
else:
    print("current dir", current_dir)
    
    
fig.write_image('plots/week_41_f1_scores_answerable.png')

current dir /Users/HansPeter/Dev/ku_local/nlp_course/src/nlp_course


In [ ]:
# look at wrong answers

answerable_preds = finetuned_preds[answerable_idxs][~perf_finetuned_answerable['exact_match_idxs']]
answerable_labels = finetuned_labels[answerable_idxs][~perf_finetuned_answerable['exact_match_idxs']]

answerable_f1_scores = perf_finetuned_answerable['f1_scores'][~perf_finetuned_answerable['exact_match_idxs']]



for pred, label, f1 in zip(answerable_preds, answerable_labels, answerable_f1_scores):
    print(f'pred:{pred}, label: {label}, f1: {f1}'.replace('\n', ' '))




In [ ]:
pred